In [1]:
"""
A first implementation of Bag of Little Bootstraps. To be cleaned and updated later.

Future possible improvements
    - vectorize computations for increased speed
"""

import numpy as np

In [2]:
def import_data():
    # First test is just averaging a random numpy array
    return np.random.rand(500000000)*100

data = import_data()
print data

[ 37.50051899  98.89092447  88.68672476 ...,  58.15618765  67.65022582
  40.39713016]


In [3]:
def calculate_average(data_array):
    return np.mean(data_array)

%timeit -n 1 -r 1 print calculate_average(data)

50.0010612009
1 loop, best of 1: 1.73 s per loop


In [4]:
#### Define variables ######
n = data.shape[0] # Size of data
b = int(n**(0.6)) # subset size
s = 1 # Number of sampled subsets
r = 20 # Number of Monte Carlo iterations

In [5]:
def calculate_avg_statistic(original_data,sample,indices):
    """
    Calculates mean according to arguments below
    
    Args:
        original_data:  original data
        sample:         multinomial sample. contains number of times that each of the b samples
                        is sampled
        indices:        indices of the data used from the original data
    """
    new_data = original_data[indices]
    return np.sum(np.multiply(sample,new_data))/n
    
calculate_avg_statistic(data,np.random.multinomial(n,pvals=[1.0/b for i in range(b)]),\
                       np.random.randint(n,size=b))

50.036407570010027

In [6]:
# TODO check algorithm correctness again
# TODO find more ways to speed up code
    # could use iterators instead of for loops
    # could use Cython, Pypy
# It still seems like the overhead of the function itself is quite low, the big thing is just
# calculating the score

def blb_main(data_array,calculate_statistic):
    subsample_estimate_sum = 0.0
    pval_array = np.ones(b)/float(b)
    # Randomly sample subset of indices
    idx = np.random.randint(low=0,high=n,size=(s,b))
    for j in range(s):
        # Approximate the measure
        monte_carlo_estimate_sum= 0.0
        multinomial_sample = np.random.multinomial(n,pvals=pval_array,size=r)
        for k in range(r):
            monte_carlo_estimate_sum += calculate_statistic(data_array,multinomial_sample[k,]\
                                                            ,idx[j,])
        subsample_estimate_sum += monte_carlo_estimate_sum/float(r)
    
    return subsample_estimate_sum/float(s)
            
    
%timeit -n 3 -r 3 print blb_main(data,calculate_avg_statistic)

49.9741745039
49.9628180205
50.046921494
50.0204673093
50.0315773615
50.0164464071
49.922004488
50.0470895312
50.1184782376
3 loops, best of 3: 483 ms per loop


In [7]:
n**(0.7)

1228228.0261157893

In [8]:
# OLD VERSION ####

# def blb_main(data_array,calculate_statistic):
#     subsample_estimate_sum = 0.0
#     pval_array = [1.0/b for i in range(b)]
#     for j in range(s):
#         # Randomly sample subset of indices
#         idx = np.random.randint(n,size=b)
#         # Approximate the measure
#         monte_carlo_estimate_sum= 0.0
#         for k in range(r):
#             multinomial_sample = np.random.multinomial(n,pvals=pval_array)
# #             print multinomial_sample
#             monte_carlo_estimate_sum += calculate_statistic(data_array,multinomial_sample,idx)
#         subsample_estimate_sum += monte_carlo_estimate_sum*1.0/r
    
#     return subsample_estimate_sum*1.0/s
            
    
# %time print blb_main(data,calculate_avg_statistic)